<div>
<img src="./images/sunpy_logo.png" width="500" align="left"/>
</div>

# 2. Data Containers

Now we have seen how we can search for and download data - lets now look at how we can read this data in.

SunPy provides core data type classes that are designed to provide a consistent interface across data types (timeseries and images) as well as data sources from numerous instruments and observations. They handle all of the manipulation necessary to read data in from mission-specific files. The two main datatypes in SunPy are

1. `TimeSeries`  
2. `Map`

In [ ]:
import sunpy.map
import sunpy.timeseries
from sunpy.coordinates import frames
from sunpy.time import parse_time
import sunpy.data.sample

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.visualization import AsymmetricPercentileInterval, ImageNormalize, LogStretch

import matplotlib.pyplot as plt
from matplotlib import colors
import glob
import numpy as np

## 2.1 TimeSeries

The structure of a [`TimeSeries`](https://docs.sunpy.org/en/stable/guide/data_types/timeseries.html) consists of times and measurements and the underlying structure is that of a `pandas.DataFrame`. 

SunPy TimeSeries supports time-series data from a wide range of solar-focused instruments. `TimeSeries` can either be created manually or from source files that are currently supported. If a supported file is passed to `TimeSeries` it will automatically detect its source and its instrument-specific meta data will be loaded. 

Lets create a timeseries from out sample data which is X-ray flux from the GOES X-ray Sensor Data. This data file was downloaded locally in the steps previously! 

In [ ]:
xrs = sunpy.timeseries.TimeSeries('./XRS/sci_xrsf-l2-flx1s_g16_d20220402_v2-2-0.nc')

In [ ]:
xrs.plot()

### Inspect the `TimeSeries`

Lets now inspect the `TimeSeries` and get at the data. A `TimeSeries` holds data as well as meta data and unit data.

In [ ]:
xrs.units

In [ ]:
xrs.meta

The `TimeSeries` object can also be converted to other formats like a `pandas.DataFrame` or an `astropy.Table` object

In [ ]:
xrs.to_dataframe()

In [ ]:
xrs.to_table()

there are also a number of attributes on each `TimeSeries` derived from the data/metadata.

In [ ]:
xrs.columns

In [ ]:
xrs.observatory

## Manipulating the timeseries data

We can manipulate the timeseries, such as truncating (slicing) the data over a certain time period

In [ ]:
xrs.truncate("2022-04-02 12:00", "2022-04-02 17:00").plot()

you can also convert to a `pandas.DataFrame` and then use the functionality there such as resampling etc

In [ ]:
xrs_df = xrs.to_dataframe()

In [ ]:
xrs_df[["xrsa", "xrsb"]].plot()
plt.yscale("log")
plt.axvline(parse_time("2022-04-02 14:00").datetime)
plt.ylim(1e-9, 1e-4)

In [ ]:
xrs_df_resample = xrs_df.resample("60s").mean()

In [ ]:
xrs_df_resample[["xrsa", "xrsb"]].plot()
plt.yscale("log")

## Solar Orbiter timeseries example

You can also pass a list of files to timeseries, and uses the `concatenate` keyword to create one continous timeseries. 

In [ ]:
mag_files = glob.glob("./MAG/*.cdf"); mag_files.sort()

mag_solo = sunpy.timeseries.TimeSeries(mag_files[0:3], concatenate=True)


In [ ]:
mag_solo.columns

In [ ]:
mag_solo.plot(columns=['B_RTN_0', 'B_RTN_1', 'B_RTN_2'])

# 2.2 Map
The sunpy [`Map`](https://docs.sunpy.org/en/stable/guide/data_types/maps.html) class provides the data type structure to store 2-dimensional data associated with a coordinate system.  This allows users to store and manipulate images of the Sun and the heliosphere.

The result of a call to Map will be either a `GenericMap` object, or a subclass of `GenericMap` which either deals with a specific type of data, e.g. `AIAMap` or `LASCOMap` (see sunpy.map Package to see a list of all of them), or if no instrument matches, a 2D map GenericMap.

Maps from all instruments are created using the `sunpy.map.Map` 'factory'. This class takes a wide variety of map-like inputs, for one or more maps and returns you one or many maps. All maps, irrespective of the instrument, behave the same and expose the same functions and properties, however, depending on the instrument different metadata might be read or corrections made.

In [ ]:
aia_file = sunpy.data.sample.AIA_171_IMAGE

In [ ]:
aia_file

In [ ]:
aia_map = sunpy.map.Map(aia_file)

In [ ]:
type(aia_map)

We can easily visualize a map after loading it using the quicklook functionality.

In [ ]:
aia_map

`Map` provides customized loaders for a number of different instruments, however, if the data file follows the FITS data standards for coordinate information etc then map should be able to read it by default.

## Attributes of Map

`Map` provides a common interface to most 2D imaging solar datasets and provides several useful pieces of metadata. As mentioned in the intro slide, `Map` is a container for holding your data and metadata (usually from the FITS header) together.

The `.meta` and `.data` attributes provide access to the metadata and underlying array of image data, respectively.


In [ ]:
aia_map.data

In [ ]:
aia_map.meta

In [ ]:
aia_map.wcs



However, this metadata can be terse, non-homogeneous, and sometimes difficult to parse. `Map` provides several attributes derived from the underlying raw metadata that expose a uniform interface to the metadata for each map.


In [ ]:
aia_map.wavelength

In [ ]:
aia_map.rsun_meters

In [ ]:
aia_map.processing_level

In [ ]:
aia_map.unit

In [ ]:
aia_map.quantity

### Coordinate Information

Each `Map` also exposes information about which coordinate system the image was taken in, including the location of the spacecraft that recorded that observation.

`sunpy` leverages and extends the powerful astropy coordinate framework that we heard about in the previous tutorial. Additionally, we'll talk more about the sunpy.coordinates subpackage in the next notebook and show some neat examples.

For each `Map`, we can easily access what coordinate frame the observation cooresponds to.


In [ ]:
aia_map.coordinate_frame

In [ ]:
aia_map.observer_coordinate

### `Map` and WCS (World Coordinate System)

The world coordinate system (WCS) formalizes provides us a framework for transforming between pixel and world coordinates. The functionality to deal with WCS within sunpy is from the `astropy` package.



In [ ]:
aia_map.wcs

In [ ]:
type(aia_map.wcs)


# World and Pixel Coordinates (Important!)

We can convert between the world coordinates (arcsec) to pixel coordinates using the `world_to_pixel` method on map which takes a `SkyCoord` and then returns the pixel coordinate. Similarly we can find the world coordinate to the pixel (or array) index. This is done with the `pixel_to_world` method. Lets first look at finding the array (pixel) index for the center of the Sun (0, 0) arcsec:



In [ ]:
aia_map.world_to_pixel(SkyCoord(0*u.arcsec, 0*u.arcsec, frame=aia_map.coordinate_frame))

In [ ]:
aia_map.pixel_to_world(0*u.pix, 0*u.pix)

## Visualization of `Map`

### Plotting a map

In [ ]:
fig = plt.figure()
aia_map.plot()

In [ ]:
fig = plt.figure()
aia_map.plot(clip_interval=[1, 99.9]*u.percent)
aia_map.draw_limb()
aia_map.draw_grid(color='w')

In [ ]:
fig = plt.figure()
aia_map.plot(cmap="viridis", clip_interval=[1, 99.5]*u.percent)

## Inspecting and Manipulating the data

In [ ]:
aia_map.data.shape

In [ ]:
aia_map.data[0, 3]

In [ ]:
print("\n Mean:", aia_map.data.mean(), "\n Max:", aia_map.data.max(), 
      "\n Min:", aia_map.data.min(),  "\n Std:", aia_map.std())

You can also perform arithimtic to the data from the maps. To do this you have to use units

In [ ]:
new_aia_map = aia_map + 10*u.ct

### Rotate a map

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned. In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image

In [ ]:
aia_map_rot = aia_map.rotate(missing=aia_map.min())

In [ ]:
aia_map_rot.plot(clip_interval=[5, 99.9]*u.percent)

In [ ]:
aia_map_rot = aia_map.rotate(angle=30*u.deg)#, missing=aia_map.min())

In [ ]:
aia_map_rot.plot(clip_interval=[0.1, 99.9]*u.percent)

# Crop a map

We commonly want to pare down our full field-of-view to a particular region of interest.
With a map, we can do this using the `submap` method.

To crop a map, we can pass either a SkyCoord (i.e. a coordinate in space), or in pixel space (i.e. by passing pixel coordinates).

We can specify the region of our submap using world coordinates as specified by a `SkyCoord`.
These coordinates can be specified in different coordinate systems and still should work (e.g. helioprojective or heliograhic stonyhurst)


In [ ]:
bottom_left = SkyCoord(-300*u.arcsec, 20*u.arcsec, frame=aia_map.coordinate_frame)
top_right = SkyCoord(390*u.arcsec, 650*u.arcsec, frame=aia_map.coordinate_frame)

In [ ]:
submap = aia_map.submap(bottom_left, top_right=top_right)

In [ ]:
fig = plt.figure()
submap.plot(clip_interval=[1, 99.9]*u.percent)

In [ ]:
fig = plt.figure(figsize=(11, 5))
ax1 = fig.add_subplot(1,2,1,projection=aia_map)
aia_map.plot(axes=ax1, clip_interval=(0.1, 99.99)*u.percent)

aia_map.draw_quadrangle(bottom_left, 
                        top_right=top_right, 
                        axes=ax1)

ax2 = fig.add_subplot(1,2,2,projection=submap)
submap.plot(clip_interval=(0.5, 99.95)*u.percent)

We can also crop a map by passing a bottom left and a width and a height, for example:

In [ ]:
submap = aia_map.submap(bottom_left, width=600*u.arcsec, height=700*u.arcsec)
submap.plot(clip_interval=[5, 99.9]*u.percent)

We can also crop a map by passing a bottom left and a width and a height in pixel coordinates. When specifying pixel coordinates, they are specified in Cartesian order not in numpy order. So, for example, the `bottom_left=` argument should be `[left, bottom]`.

In [ ]:
submap = aia_map.submap([350, 520]*u.pix, width=350*u.pix, height=250*u.pix)
submap.plot(clip_interval=[5, 99.9]*u.percent)

## Resample a map

In [ ]:
aia_map.resample([40, 40] * u.pixel).plot()

In [ ]:
aia_map.resample([256, 256] * u.pixel).plot(clip_interval=[5, 99.9]*u.percent)

In [ ]:
aia_resampled = aia_map.resample([256, 256] * u.pixel)

In [ ]:
aia_resampled.data.shape

# Sequence of Maps

A MapSequence is an ordered list of maps. By default, the maps are ordered by their observation date, from earliest to latest date. Lets use a time list of maps that we have already downloaded and generate them into a movie. 

In [ ]:
aia_files = [sunpy.data.sample.AIA_193_CUTOUT01_IMAGE,
             sunpy.data.sample.AIA_193_CUTOUT02_IMAGE,
             sunpy.data.sample.AIA_193_CUTOUT03_IMAGE,
             sunpy.data.sample.AIA_193_CUTOUT04_IMAGE,
             sunpy.data.sample.AIA_193_CUTOUT05_IMAGE,]

In [ ]:
aia_files

In [ ]:
aia_map_sequence = sunpy.map.Map(aia_files, sequence=True)

In [ ]:
ani = aia_map_sequence.plot(cmap=aia_map_sequence[0].plot_settings['cmap'],
                            norm=ImageNormalize(vmin=1, vmax=1e4,
                                        stretch=aia_map_sequence[0].plot_settings['norm'].stretch))
ani.save('aia-maps-seq.mp4', fps=15, dpi=300)

## Running difference of maps

In [ ]:
aia_diff_map = (aia_map_sequence[1] - aia_map_sequence[0].quantity)


In [ ]:
aia_diff_map.plot(norm=colors.Normalize(), vmin=-500, vmax=500)
aia_diff_map.draw_limb()

In [ ]:
aia_diff_deq = sunpy.map.Map(
    [m - prev_m.quantity for m, prev_m in zip(aia_map_sequence[1:], aia_map_sequence[:-1])],
    sequence=True
)

In [ ]:
ani = aia_diff_deq.plot( title='Running Difference', 
                         norm=colors.Normalize(vmin=-500, vmax=500), cmap='Greys_r')
ani.save('aia-maps-seq.mp4', fps=15, dpi=300)

## WCS axes and plotting

SunPy map uses the [`astropy.visualization.wcsaxes`](https://docs.astropy.org/en/stable/visualization/wcsaxes/index.html#module-astropy.visualization.wcsaxes) module to represent world coordinates. 

Using WCSAxes is very powerful but has important concepts to think about:

 * **`world`** coordinates refer to the coordinates of the coordinate system - i.e. arcsec, degrees!
 * **`pixel`** coordinates refer to the array index of the data! i.e. data[10] etc. However, the convention of pixel axes is the opposite to numpy arrays - i.e. to [x, y], rather the [y, x]
 
 
When plotting on WCSAxes it will by default plot in pixel coordinates, you can override this behavior and plot in `world` coordinates by getting the transformation from the axes with `ax.get_transform('world')`. We will use some of these examples below. Its also important to note that when using the `world` coordinates these have to be in **degrees** so make sure to convert arcsec's to degrees.


In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection=aia_map)  

# plot the map
aia_map.plot(clip_interval=[0.5, 99.99]*u.percent)
aia_map.draw_limb()
aia_map.draw_grid()

# plot in pixel coordinates
ax.plot(200, 200, marker='o', color="b",  label="Pixel coord")

# plot in world coordinates
ax.plot((200*u.arcsec).to(u.deg), (200*u.arcsec).to(u.deg),
        transform=ax.get_transform('world'), 
        marker='o',color="g", label="World Coord")

ax.legend()



You can also plot SkyCoords on a Map. Importantly, this can be done with ax.plot_coord and the coordinate does not need to be transformed to the same coordinate frame as the map - it is done automatically if it can be


In [ ]:
coord1 = SkyCoord(200*u.arcsec, -500*u.arcsec, frame=aia_map.coordinate_frame)
coord2 = SkyCoord(20*u.deg, 30*u.deg, frame=frames.HeliographicStonyhurst)

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(projection=aia_map)
aia_map.plot(axes=ax, clip_interval=[0.5, 99.99]*u.percent)

aia_map.draw_grid(axes=ax)

ax.plot_coord(coord1, marker='o', ms=10, color='b')
ax.plot_coord(coord2, marker='x', ms=10, color='b')



In [ ]:
pixel_pos = np.argwhere(aia_map.data == aia_map.data.max()) * u.pixel
hpc_max = aia_map.pixel_to_world(pixel_pos[:, 1], pixel_pos[:, 0])

In [ ]:
pixel_pos

In [ ]:
plt.imshow(aia_map.data, norm=colors.LogNorm(), origin="lower")
plt.scatter(808, 362, color='r')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=aia_map)
aia_map.plot(clip_interval=[5, 99.9]*u.percent)
ax.plot_coord(hpc_max, marker='o', color='b')
ax.scatter(362, 808, color='r', marker='x')

### Lets find the maximum pixel of the AIA map and plot it

In [ ]:
pixel_pos = np.argwhere(aia_map.data == aia_map.data.max()) * u.pixel
hpc_max = aia_map.pixel_to_world(pixel_pos[:, 1], pixel_pos[:, 0])

fig = plt.figure()
ax = fig.add_subplot(projection=aia_map)
aia_map.plot(clip_interval=[5, 99.9]*u.percent)
ax.plot_coord(hpc_max, marker='x')

## Other functionality on Map

There's lots of other imaginative things you can do with sunpy.map.GenericMap and with the infrastructure of sunpy. We recommend checking out our documentation more and the example gallery

## Take slice across the Sun or region of interest

In [ ]:
line_coords = SkyCoord([-1200, 1200], [500, 500], unit=(u.arcsec, u.arcsec),
                       frame=aia_map.coordinate_frame)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection=aia_map)
aia_map.plot(clip_interval=[5, 99.9]*u.percent)
ax.plot_coord(line_coords)

In [ ]:
intensity_coords = sunpy.map.pixelate_coord_path(aia_map, line_coords)

In [ ]:
intensity = sunpy.map.sample_at_coords(aia_map, intensity_coords)

In [ ]:
angular_separation = intensity_coords.separation(intensity_coords[0]).to(u.arcsec)

In [ ]:
plt.plot(angular_separation, intensity)
plt.ylabel("Intensity (DN)")
plt.xlabel("Distance along path (arcsec)")